# Ollama API Testing

This notebook tests the connection to the Ollama container and verifies that the llama3.2 model is working properly.

## Steps:
1. Test basic connectivity to Ollama
2. Import required libraries  
3. Set up API constants
4. Check if model is available (and pull if needed)
5. Test the model with a simple chat message

In [ ]:
# imports
import requests

response = requests.get("http://ollama:11434/api/tags")
print(response.json())


In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://ollama:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Test chatting with the LLM
messages = [
    {
        "role": "user",
        "content": "Hello! Can you tell me a short joke about programming?"
    }
]

payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

print("Sending message to llama3.2...")
try:
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    if response.status_code == 200:
        result = response.json()
        print("\n🤖 Response from llama3.2:")
        print("-" * 40)
        print(result['message']['content'])
        print("-" * 40)
    else:
        print(f"❌ API call failed with status: {response.status_code}")
        print(f"Response: {response.text}")
except Exception as e:
    print(f"❌ Error making API call: {e}")
    print("Make sure the Docker containers are running!")

In [ ]:
# Check if model is available and pull if needed
print("Checking if llama3.2 model is available...")

try:
    response = requests.get("http://ollama:11434/api/tags")
    if response.status_code == 200:
        models = response.json()
        model_names = [model['name'] for model in models.get('models', [])]
        print(f"📋 Available models: {model_names}")
        
        # Check if llama3.2 is available (could be 'llama3.2:latest' or just 'llama3.2')
        llama_available = any('llama3.2' in name for name in model_names)
        
        if not llama_available:
            print("⬇️ Model llama3.2 not found. Pulling model...")
            # Pull the model using the API
            pull_response = requests.post("http://ollama:11434/api/pull", 
                                        json={"name": "llama3.2"})
            if pull_response.status_code == 200:
                print("✅ Model pulled successfully!")
            else:
                print(f"❌ Failed to pull model: {pull_response.status_code}")
        else:
            print("✅ Model llama3.2 is already available!")
    else:
        print(f"❌ Failed to check models: {response.status_code}")
        
except Exception as e:
    print(f"❌ Error checking models: {e}")
    print("Make sure the Docker containers are running!")